In [ ]:
import numpy as np
import os
from os.path import isfile


import librosa
import librosa.display
import matplotlib.pyplot as plt
%matplotlib inline

### Load song

In [ ]:
dict_genres = {'Electronic':0,  'Folk':1,  'Pop' :2, 'Instrumental':3 }


reverse_map = {v: k for k, v in dict_genres.items()}
#print(reverse_map)

In [ ]:
import IPython.display as ipd
files = os.listdir(os.path.join("predict"))
#print(files)

time=420
freq=128
arr_specs=list()
#X_spect = np.empty((0, time, freq))
X_spect = np.empty((0, time))
songs=[]
for fname in files:
  print(fname)
  count=0
  for index in range(18):
  
    y,sr=librosa.load(os.path.join("predict",fname),duration=10,offset=index*10)
    spect = librosa.feature.melspectrogram(y=y, sr=sr,n_fft=2048, hop_length=512)
    spect = librosa.power_to_db(spect, ref=np.max)
    spect = spect[:freq, :time]
    spect=spect.T
    songs.append(y)
    arr_specs.append(spect)  
    X_spect=np.array(arr_specs) 
  
  
print(X_spect.shape)  

In [ ]:
from keras.models import load_model

weights_path = 'models/crnn/weights.best.h5'
model_crnn = load_model(weights_path)

weights_path = 'models/parallel/weights.best.h5'
model_prcnn = load_model(weights_path)

weights_path = 'models/cnn/weights.best.h5'
model_cnn = load_model(weights_path)


In [ ]:
x_test=X_spect

print(x_test.shape)

In [ ]:
y_pred_crnn = model_crnn.predict(x_test)
#print(y_pred.shape)
x_test = x_test.reshape(x_test.shape[0],x_test.shape[1],x_test.shape[2],1)
y_pred_prcnn = model_prcnn.predict(x_test)
y_pred_cnn = model_cnn.predict(x_test)

In [ ]:
res_list=[]
res_one=[]
#res_print[]
def pred(y_pred):
  total=0
  result={}
  count=0
  #result=[]
  for j in y_pred:
    total+=j
  for j in y_pred:  
    result[reverse_map[count]]=j/total 
    count=count+1
  labels=sorted(result.items(), key=lambda x: x[1], reverse=True)
  for x in labels[:2:]:
     res_list.append(x[0])
  #for x in labels[0:]:
    #print(x)  
  for x in labels[:1:]:
    res_one.append(x[0])

### CNN Prediction

In [ ]:
print("CNN prediction")
for i in y_pred_cnn:
  pred(i)
  #print("\n")

In [ ]:
import collections
from collections import Counter

x=list(Counter(res_list).keys())
y=list(Counter(res_list).values())

result_2=dict(zip(x, y))
result_1=dict(zip(Counter(res_one).keys(), Counter(res_one).values()))

print("Top 2 genres predicton:",result_2)
print("Top one genre prediction",result_1)

### CRNN Prediction

In [ ]:

res_list=[]
res_one=[]

print("CRNN prediction")
for i in y_pred_crnn:
  pred(i)
  #print("\n")

### Top 1,2 Genres Prediction

In [ ]:
import collections
from collections import Counter

x=list(Counter(res_list).keys())
y=list(Counter(res_list).values())

result_2=dict(zip(x, y))
result_1=dict(zip(Counter(res_one).keys(), Counter(res_one).values()))

print("Top 2 genres predicton:",result_2)
print("Top one genre prediction",result_1)

### Plot Top 2 Genres Prediction

In [ ]:
import matplotlib.pyplot as plt
D = result_2
plt.figure(figsize=(8,8))
plt.bar(range(len(D)), list(D.values()), align='center')
plt.xticks(range(len(D)), list(D.keys()))
plt.title("CRNN Top-2 prediction")
plt.show()

### PRCNN Prediction

In [ ]:
print("PRCNN prediction")
res_list=[]
res_one=[]
for i in y_pred_prcnn:
  pred(i)
  #print("\n")

### Top 1,2 Genres Prediction

In [ ]:
from collections import Counter
import collections
x=list(Counter(res_list).keys())
y=list(Counter(res_list).values())

result_2=dict(zip(x, y))
result_1=dict(zip(Counter(res_one).keys(), Counter(res_one).values()))


print("Top 2 genres predicton:",result_2)
print("Top one genre prediction",result_1)

### Plot Top 2 Genres Prediction

In [ ]:
import matplotlib.pyplot as plt
D = result_2
plt.figure(figsize=(8,8))
plt.bar(range(len(D)), list(D.values()), align='center')
plt.xticks(range(len(D)), list(D.keys()))
plt.title("PRCNN Top-2 prediction")
plt.show()